# TRANSFORMS

Thông thường khi load data vào thì nó sẽ không theo một dạng chuẩn nào đó, muốn data về dạng mà ta mong muốn thì cần p qua bước transform. `torchvision` cung cấp các build in `transform` để pass vào `transform` và `target_transform` khi load data.

Ví dụ:


In [2]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(
    root="../../data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

Ở ví dụ trên ta có sử dụng `ToTensor` để transform input từ `numpy.ndarray` sang `tensor` của Pytorch.

`Lambda` transform dùng để apply cho các `lambda` function

Với

```python
lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)
```
Thực chuyển giá trị target thành `one-hot encoded` tenser theo các bước: 
- tạo zero tensor có size là 10 (số labels trong dataset)
- `scatter_` sẽ assign `value=1` cho element trong `tensor 10` có `index = labels trong dataset`

## Custom dataset và implement Custom transform, multiple transform

In [1]:
import torchvision
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch



# Custom Dataset
class WineDataset(Dataset):
    def __init__(self, transform=None):
        # download data:
        xy = np.loadtxt("../../data/wine/wine.csv",
                        delimiter=',',
                        dtype=np.float32, 
                        skiprows = 1)
        # Not conver, transform data here
        self.x = xy[:,1:]
        self.y = xy[:,[0]]
        self.n_samples = xy.shape[0]
        self.transform = transform

    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)
        return sample

    def __len__(self):
        return self.n_samples     


# Custom transform
class ToTensor:
    def __call__(self, sample):
        inputs, labels = sample
        return torch.from_numpy(inputs), torch.from_numpy(labels)    

class MulTransform:
    def __init__(self,factor):
        self.factor = factor

    def __call__(self, sample):
        inputs, labels = sample
        inputs *=self.factor
        return inputs, labels

dataset = WineDataset(transform=ToTensor())
input, label = dataset[0]
print(type(input), type(label))
print("Before transform:")
print(input)
print(label)

# using composed transform
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(3)])
dataset = WineDataset(transform=composed)
input, label = dataset[0]

print("After transform:")
print(type(input), type(label))
print(input)
print(label)


<class 'torch.Tensor'> <class 'torch.Tensor'>
Before transform:
tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03])
tensor([1.])
After transform:
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([4.2690e+01, 5.1300e+00, 7.2900e+00, 4.6800e+01, 3.8100e+02, 8.4000e+00,
        9.1800e+00, 8.4000e-01, 6.8700e+00, 1.6920e+01, 3.1200e+00, 1.1760e+01,
        3.1950e+03])
tensor([1.])
